In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,
    confusion_matrix, classification_report
)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from keras import Input, Model
from keras.layers import (
    Embedding, Bidirectional, LSTM, Dropout, BatchNormalization,
    Conv1D, MaxPooling1D, GlobalMaxPooling1D, Flatten, Dense, GRU
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import gc
#warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
splits = {'train': 'tamil_train_cleaned.csv', 'val': 'tamil_val_cleaned.csv','test':'tamil_test_cleaned.csv'}
train_df = pd.read_csv("hf://datasets/kcrl/Shared_task_Tamil/" + splits["train"])
val_df = pd.read_csv("hf://datasets/kcrl/Shared_task_Tamil/" + splits["val"])
test_df = pd.read_csv("hf://datasets/kcrl/Shared_task_Tamil/" + splits["test"])

In [3]:
with pd.option_context('display.max_colwidth',0):
    display(train_df.sample(n=10))

,content,labels
2450,காவலில் விசாரிப்பு!\n\ncurrentaffairs breakingnews tamilnadu tamilnews tamilagaarasiyal cinemaupdate mkstalin dmk aiadmk newsupdate\ntodaypolitics todaynews topnews,Opinionated
1730,newsupdate | ராகுல் காந்தியை ஸ்டைலாக வரவேற்ற முதல்வர் \n\nclick link: \n\nnewstamil24x7 | coimbatore | mkstalin | dmk | rahul_gandhi | congressparty | elections2024,Positive
11,"""மோடி வருவாரு ஓடிப் போயிருவாரு""\n\nmodi bjp seeman ntk tamilnews",Sarcastic
1482,"அன்பின் உயரம் ""எளிமை"" !\nஅண்ணனுக்காக தானே கடைக்கு போய் இனிப்பு வாங்குவது \n\nrahulgandhi mkstalin brotherhood \nvote4indiaalliance coimbatore\n",Positive
3100,"உங்களுக்கு கூடும் கூட்டம் காசுக்காகவும், குவாட்டருக்காகவும் கூடும். \n\nஎங்களுக்கு கூடும் கூட்டம் எங்களின் கொள்கைக்காக கூடும் கூட்டம். காசு, குவாட்டர் எதுவும் கிடையாது. இது தானா சேர்ந்த கூட்டம் \n\nntk_symbol_mike seeman \nசீமானின்_சின்னம்_ஒலிவாங்கி",Negative
288,"அதிமுக செய்யும் அட்டூழியத்தை மறைத்து அதை திமுக மேல பழியை போடவும்,\nராஜபக்ஷே செய்த கொலைகளை மறைத்து அதை திமுக மேல பழி போடவும் முதலாளிகளால் வளர்க்கப்படும் வேட்டை நாய்கள் தான் ntk",Positive
452,பரப்புரை\n\ntamilnadu | thoothukudi | kanimozhikarunanidhi | dmk | mkstalin | loksabaelection2024 | parliamentelection2024 | elections2024 | electionswithnews7tamil | news7tamil | news7tamilupdates,Opinionated
2933,ஆயிரம் ரூபாய் கொடுப்பது சாதனை இல்லை; ஆயிரம் ரூபாய் பெறவேண்டிய நிலை இல்லை என்பதை உருவாக்குவது தான் சாதனை - சீமான் \n\n | seeman loksabaelection2024 elections2024 tamilnews,Neutral
2849,"தி.மு.க,வுக்கு எதிராக பதிவு;\nசாஸ்திரிகள் மீது சரமாரி தாக்குதல்!\nshastrisitharaman | dmk | dmkattack | elections2024\n",Negative
1727,இந்து தானே என்று சொல்லி வேறு மாநிலத்தில் தேர்தலில் வெற்றி பெற முடியுமா!\n\nseeman\nseeman4tn\nntk\nntk4tn\nelections2024\nloksabhaelection2024\ntnpolitics\nசீமான்\nநாம்தமிழர்கட்சி,Positive


In [4]:
train_df.labels.unique()

array(['Neutral', 'Substantiated', 'Opinionated', 'Positive', 'Sarcastic',
       'Negative', 'None of the above'], dtype=object)

In [5]:
labeling = {
    'Neutral': 0,
    'Substantiated': 1,
    'Opinionated': 2,
    'Positive' : 3,
    'Sarcastic': 4,
    'Negative': 5,
    'None of the above': 6
}
train_df['labels'] = train_df['labels'].apply(lambda x : labeling[x])
test_df['labels'] = test_df['labels'].apply(lambda x : labeling[x])
val_df['labels'] = val_df['labels'].apply(lambda x : labeling[x])

In [6]:
with pd.option_context('display.max_colwidth',0):
    display(train_df.sample(n=5))

,content,labels
1699,அம்மா' என குறிப்பிட்டுவாக்கு சேகரித்த பிரதமர் மோடி | pm modi | bjp | election campaign \n\nnewstamil24x7 admk jayalalitha pmmodi bjp electioncampaign elections2024 parliamentelection2024 loksabhaelection2024 vellore,4
888,தமிழ்நாட்டிற்கு மோடி என்ன செய்தார் ? விவசாயிகளுக்கு மோடி என்ன செய்தார் ? \nnarendramodi dmkfailstn annamalai4coimbatore,2
2308,செஞ்சிட்டோமே! செஞ்சிட்டோமே! \nமக்களை வைச்சு செஞ்சிட்டோமே ! \n\nதேர்தல் ஸ்பெஷல் பாடல் வெளியீடு\n\nமுழு பாடல் - நாளை காலை 8 மணிக்கு,4
3600,லகுனத்தை தரிக்கிற \n“ழ” கரத்தை சேர்த்தாம்\n\nதமிழ்நாட்டு மக்களின் சின்னம் \nஒலிவாங்கி\nvoteforntk \nசீமானின்_சின்னம்_ஒலிவாங்கி \nntk_symbol_mike \nநன்றி,5
1402,பாஜக மிரட்டலுக்கு பயப்படாத திமுக dmkfailstn tamilnadu bjp4india,5


In [7]:
from sklearn.utils import resample

# Set the target number of instances for minority classes
target_minority_count = 200

for minority_class in [0,1,3,5,6]:
    minority_indices = train_df[train_df['labels'] == minority_class].index
    minority_data = train_df.loc[minority_indices]

    minority_oversampled = resample(
        minority_data,
        replace=True,
        n_samples=target_minority_count,
        random_state=42
    )

    train_df = pd.concat([train_df, minority_oversampled], ignore_index=True)

In [8]:
X_train = train_df['content']
y_train = train_df['labels']
X_val = val_df['content']
y_val = val_df['labels']
X_test = test_df['content']
y_test = test_df['labels']

In [9]:
# Tokenizing the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab=tokenizer.word_index

In [10]:
train_sequence = tokenizer.texts_to_sequences(X_train)
val_sequence = tokenizer.texts_to_sequences(X_val)
test_sequence = tokenizer.texts_to_sequences(X_test)

In [11]:
maxlen = 120
train_data = pad_sequences(train_sequence, maxlen=maxlen,  padding='post')
val_data = pad_sequences(val_sequence, maxlen=maxlen,  padding='post')
test_data = pad_sequences(test_sequence, maxlen=maxlen,  padding='post')

In [12]:
train_labels = y_train
val_labels = y_val
test_labels = y_test

In [13]:
train_labels_reshaped = np.expand_dims(train_labels, axis=-1)
val_labels_reshaped = np.expand_dims(val_labels, axis=-1)
test_labels_reshaped = np.expand_dims(test_labels, axis=-1)

In [14]:
vocab_size = len(tokenizer.word_index) + 1
input_shape = (maxlen,)

In [19]:
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dropout, BatchNormalization, Dense, Flatten
from keras.regularizers import l2

# Define the embedding layer
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)

# Define the input layer
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define the Bidirectional LSTM layers
lstm_layer1 = LSTM(units=64, return_sequences=True)
lstm_layer2 = LSTM(units=32, return_sequences=True)

# Apply the first LSTM layer
lstm_output1 = lstm_layer1(embedded_email)

# Apply dropout after the first LSTM layer
dropout1 = Dropout(0.3)(lstm_output1)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Apply the second LSTM layer
lstm_output2 = lstm_layer2(bn1)

# Flatten the output
flatten_output = Flatten()(lstm_output2)

# Add a dense layer for multiclass classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.01))(flatten_output)

# Create the model
lstm_model = Model(inputs=email_input, outputs=output)

# Compile the model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
lstm_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 120, 300)            │       5,125,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 120, 64)             │          93,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 120, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 120, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 120, 32)             │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 3840)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │          26,887 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,258,499 (20.06 MB)

 Trainable params: 5,258,371 (20.06 MB)

 Non-trainable params: 128 (512.00 B)

In [20]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=lstm_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2633 - loss: 1.9530
Epoch 1: val_loss improved from inf to 1.93797, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.2645 - loss: 1.9502 - val_accuracy: 0.2647 - val_loss: 1.9380
Epoch 2/15
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6627 - loss: 1.1028
Epoch 2: val_loss improved from 1.93797 to 1.85723, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.6630 - loss: 1.1017 - val_accuracy: 0.3015 - val_loss: 1.8572
Epoch 3/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8755 - loss: 0.5189
Epoch 3: val_loss improved from 1.85723 to 1.80248, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.8753 - loss: 0.5188 - val_accuracy: 0.3088 - val_loss: 1.8025
Epoch 4/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9154 - loss: 0.3224
Epoch 4: val_loss did not improve fro

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = lstm_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Precision (Micro): 0.34558823529411764
Recall (Micro): 0.34558823529411764
F1 Score (Micro): 0.34558823529411764
Accuracy: 0.34558823529411764


In [24]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
# Define the input layer
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define the Bidirectional LSTM layers
lstm_layer1 = Bidirectional(LSTM(units=64, return_sequences=True))
lstm_layer2 =Bidirectional(LSTM(units=32, return_sequences=True))

# Apply the first LSTM layer
lstm_output1 = lstm_layer1(embedded_email)

# Apply dropout after the first LSTM layer
dropout1 = Dropout(0.3)(lstm_output1)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Apply the second LSTM layer
lstm_output2 = lstm_layer2(bn1)

flatten_output = Flatten()(lstm_output2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.01))(flatten_output)

# Create the model
bilstm_model = Model(inputs=email_input, outputs=output)

# Compile the model
bilstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
bilstm_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_4 (Embedding)              │ (None, 120, 300)            │       5,125,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 120, 128)            │         186,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 120, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 120, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 120, 64)             │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 7680)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │          53,767 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,407,875 (20.63 MB)

 Trainable params: 5,407,619 (20.63 MB)

 Non-trainable params: 256 (1.00 KB)

In [25]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bilstm_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2635 - loss: 1.9403
Epoch 1: val_loss improved from inf to 1.91337, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.2646 - loss: 1.9381 - val_accuracy: 0.2812 - val_loss: 1.9134
Epoch 2/15
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6804 - loss: 1.0833
Epoch 2: val_loss improved from 1.91337 to 1.82152, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.6807 - loss: 1.0818 - val_accuracy: 0.3033 - val_loss: 1.8215
Epoch 3/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.8738 - loss: 0.4899
Epoch 3: val_loss improved from 1.82152 to 1.81379, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.8737 - loss: 0.4898 - val_accuracy: 0.3107 - val_loss: 1.8138
Epoch 4/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9239 - loss: 0.2960
Epoch 4: val_loss did not improve fro

In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bilstm_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Precision (Micro): 0.31985294117647056
Recall (Micro): 0.31985294117647056
F1 Score (Micro): 0.31985294117647056
Accuracy: 0.31985294117647056


In [27]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define and apply the Bidirectional LSTM layer
lstm_layer = Bidirectional(LSTM(units=128, return_sequences=True))(embedded_email)

# Apply dropout after LSTM
dropout1 = Dropout(0.3)(lstm_layer)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Add CNN layers after BiLSTM
# First CNN block
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(bn1)
pool1 = MaxPooling1D(pool_size=2)(conv1)

# Second CNN block
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)

# Apply global max pooling
global_pool = GlobalMaxPooling1D()(pool2)

# Add dropout for regularization
dropout2 = Dropout(0.2)(global_pool)

# Add batch normalization
bn2 = BatchNormalization()(dropout2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.001))(bn2)

# Create the model
bilstm_cnn_model = Model(inputs=email_input, outputs=output)

# Compile the model
bilstm_cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display the model summary
bilstm_cnn_model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_5 (Embedding)              │ (None, 120, 300)            │       5,125,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 120, 256)            │         439,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 120, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 120, 256)            │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 120, 128)            │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 60, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 60, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,689,603 (21.70 MB)

 Trainable params: 5,688,963 (21.70 MB)

 Non-trainable params: 640 (2.50 KB)

In [28]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bilstm_cnn_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1624 - loss: 2.5198
Epoch 1: val_loss improved from inf to 2.35855, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.1627 - loss: 2.5183 - val_accuracy: 0.2114 - val_loss: 2.3585
Epoch 2/15
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3332 - loss: 1.8255
Epoch 2: val_loss did not improve from 2.35855
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.3349 - loss: 1.8213 - val_accuracy: 0.2151 - val_loss: 2.4976
Epoch 3/15
82/84 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6962 - loss: 0.9143
Epoch 3: val_loss improved from 2.35855 to 2.14740, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.6971 - loss: 0.9125 - val_accuracy: 0.2279 - val_loss: 2.1474
Epoch 4/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8718 - loss: 0.4685
Epoch 4: val_loss improved from 2.14740 to 2.02731, saving model to model.weights.b

In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bilstm_cnn_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Precision (Micro): 0.31066176470588236
Recall (Micro): 0.31066176470588236
F1 Score (Micro): 0.31066176470588236
Accuracy: 0.31066176470588236


In [30]:
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=300,
    input_length=maxlen,
    trainable=True  # Allow fine-tuning of embeddings
)
email_input = Input(shape=(maxlen,), name='Email_input')

# Apply the embedding layer to the input
embedded_email = embedding_layer(email_input)

# Define and apply the Bidirectional LSTM layer
lstm_layer = Bidirectional(GRU(units=128, return_sequences=True))(embedded_email)

# Apply dropout after LSTM
dropout1 = Dropout(0.3)(lstm_layer)

# Apply batch normalization
bn1 = BatchNormalization()(dropout1)

# Add CNN layers after BiLSTM
# First CNN block
conv1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(bn1)
pool1 = MaxPooling1D(pool_size=2)(conv1)

# Second CNN block
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)

# Apply global max pooling
global_pool = GlobalMaxPooling1D()(pool2)

# Add dropout for regularization
dropout2 = Dropout(0.2)(global_pool)

# Add batch normalization
bn2 = BatchNormalization()(dropout2)

# Add a dense layer for binary classification
output = Dense(7, activation='softmax', kernel_regularizer=l2(0.001))(bn2)

# Create the model
bigru_cnn_model = Model(inputs=email_input, outputs=output)

# Compile the model
bigru_cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Display the model summary
bigru_cnn_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Email_input (InputLayer)             │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_6 (Embedding)              │ (None, 120, 300)            │       5,125,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 120, 256)            │         330,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 120, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 120, 256)            │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 120, 128)            │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 60, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 60, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,580,547 (21.29 MB)

 Trainable params: 5,579,907 (21.29 MB)

 Non-trainable params: 640 (2.50 KB)

In [31]:
stopper= EarlyStopping(patience=5, monitor='val_accuracy')
checkpointer = ModelCheckpoint(filepath='model.weights.best.keras', verbose=3, save_best_only=True)

history=bigru_cnn_model.fit(
         train_data, train_labels_reshaped,
         batch_size=64,
         epochs=15,
         validation_data=(val_data, val_labels_reshaped),
         callbacks=[stopper, checkpointer],
         )

Epoch 1/15
82/84 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1712 - loss: 2.4292
Epoch 1: val_loss improved from inf to 2.29594, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.1720 - loss: 2.4244 - val_accuracy: 0.1268 - val_loss: 2.2959
Epoch 2/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3064 - loss: 1.8270
Epoch 2: val_loss did not improve from 2.29594
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.3070 - loss: 1.8256 - val_accuracy: 0.1268 - val_loss: 2.5003
Epoch 3/15
83/84 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6469 - loss: 1.0685
Epoch 3: val_loss did not improve from 2.29594
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.6473 - loss: 1.0674 - val_accuracy: 0.1268 - val_loss: 2.4054
Epoch 4/15
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.8327 - loss: 0.6043
Epoch 4: val_loss improved from 2.29594 to 2.18714, saving model to model.weights.best.keras
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/s

In [32]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get predictions (class indices for each sample)
pred = bigru_cnn_model.predict(test_data)
y_pred = np.argmax(pred, axis=1)  # Get the class with the highest probability

# Calculate metrics
precision = precision_score(y_test, y_pred, average='micro')  # Micro-average precision
recall = recall_score(y_test, y_pred, average='micro')        # Micro-average recall
f1 = f1_score(y_test, y_pred, average='micro')                # Micro-average F1 score
accuracy = accuracy_score(y_test, y_pred)                     # Accuracy

# Print results
print("Precision (Micro):", precision)
print("Recall (Micro):", recall)
print("F1 Score (Micro):", f1)
print("Accuracy:", accuracy)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Precision (Micro): 0.3382352941176471
Recall (Micro): 0.3382352941176471
F1 Score (Micro): 0.3382352941176471
Accuracy: 0.3382352941176471
